In [ ]:
import pandas as pd

df = pd.read_csv("../Trell.csv")

# removing redundant first column from the dataframe
df = df.iloc[:, 1:]

# no of unique users in the dataset
print( len(pd.unique(df["userId"])) )


In [ ]:
filter_col = [col for col in df if col.startswith('Category') | col.startswith('languageId') | col.startswith('Age') | col.startswith('gender') | col.startswith('duration_preprocessedduration') | col.startswith('target_gender') | col.startswith('duration_preprocessed_modeduration') | col.startswith('duration_preprocessed_modeduration') | col.startswith('User_Interest') | col.startswith('tier') | col.startswith('event_type') | col.startswith('categoryId') ]

filter_col += ["userId", "viewing_date_global", "trail_avgtimeSpent", "timespent_global", "completion_global", "duration_preprocessed_mode_freq", "completion_mode_score"]

print( len(filter_col))

df= df[filter_col]


In [ ]:
print( df )

In [ ]:
## handling null and empty values

In [ ]:
print( pd.isna(df).any() )

In [ ]:
for col in df.columns: 
    if col.startswith('Category') | col.startswith('duration'):
        df[col] = df[col].fillna(0)

In [ ]:
print( df.columns[df.isna().any()] )

In [ ]:
df['Age0'] = df['Age0'].fillna( df['Age0'].mean() )
df['tier0'] = df['tier0'].fillna(-1)
df['categoryId'] = df['categoryId'].fillna( df['categoryId'].mean() )